# Column Transformer Sklearn

In [5]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

In [6]:
# load dataset
df = pd.read_csv('../Dataset/covid_toy.csv')

In [7]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
42,27,Male,100.0,Mild,Delhi,Yes
44,20,Male,102.0,Strong,Delhi,No
12,25,Female,99.0,Strong,Kolkata,No
45,72,Male,99.0,Mild,Bangalore,No
79,48,Female,103.0,Mild,Kolkata,Yes


In [24]:
# Count the occurrences of each unique value in the 'cough' column
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [25]:
# Count the number of occurrences of each unique city in the 'city' column
df['city'].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [26]:
# Check for missing values in each column and count them
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [27]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(df.drop(columns='has_covid'), df['has_covid'], test_size=0.2)

In [12]:
X_train

,age,gender,fever,cough,city
89,46,Male,103.0,Strong,Bangalore
83,17,Female,104.0,Mild,Kolkata
16,69,Female,103.0,Mild,Kolkata
95,12,Female,104.0,Mild,Bangalore
98,5,Female,98.0,Strong,Mumbai
...,...,...,...,...,...
49,44,Male,104.0,Mild,Mumbai
70,68,Female,101.0,Strong,Delhi
26,19,Female,100.0,Mild,Kolkata
25,23,Male,NaN,Mild,Mumbai


# Aam Zindagi

In [13]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.transform(X_test[['fever']])

In [14]:
X_train_fever.shape

(80, 1)

In [15]:
# Ordinal Encoding
oe= OrdinalEncoder(categories=[['Mild', 'Strong']])

X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.transform(X_test[['cough']])
X_train_cough.shape

(80, 1)

In [16]:
# OHE on gender and city
ohe= OneHotEncoder(sparse_output=False, drop='first')
X_train_gender_city = ohe.fit_transform(X_train[['gender', 'city']])

# also test data
X_test_gender_city = ohe.transform(X_test[['gender', 'city']])
X_train_gender_city.shape

(80, 4)

In [17]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values

#also the test data
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values
X_train_age

array([[46],
       [17],
       [69],
       [12],
       [ 5],
       [81],
       [54],
       [79],
       [82],
       [16],
       [60],
       [47],
       [82],
       [47],
       [65],
       [81],
       [82],
       [15],
       [65],
       [80],
       [65],
       [10],
       [83],
       [20],
       [49],
       [59],
       [24],
       [80],
       [25],
       [16],
       [70],
       [31],
       [14],
       [51],
       [49],
       [34],
       [56],
       [65],
       [75],
       [60],
       [38],
       [33],
       [42],
       [71],
       [83],
       [64],
       [84],
       [55],
       [20],
       [14],
       [12],
       [71],
       [24],
       [73],
       [27],
       [48],
       [51],
       [10],
       [83],
       [18],
       [27],
       [19],
       [42],
       [25],
       [69],
       [13],
       [38],
       [34],
       [34],
       [ 5],
       [ 6],
       [64],
       [42],
       [40],
       [27],
       [44],
       [68],

In [28]:
# Combine different preprocessed feature arrays horizontally to create the final training and testing feature sets
X_train_transformed = np.concatenate((X_train_age, X_train_fever, X_train_gender_city, X_train_cough), axis=1)
X_test_transformed  = np.concatenate((X_test_age, X_test_fever, X_test_gender_city, X_test_cough), axis=1)

# Check the shapes of the transformed training and testing feature sets
X_train_transformed.shape, X_test_transformed.shape


((80, 7), (20, 7))

# Mentos Zindagi

In [19]:
from sklearn.compose import ColumnTransformer

In [30]:
# ColumnTransformer to preprocess data efficiently:
# 1. Fill missing values in 'fever' with SimpleImputer
# 2. Convert 'cough' to ordinal values ('Mild'=0, 'Strong'=1)
# 3. One-hot encode categorical columns 'gender' and 'city' (drop first to avoid multicollinearity)
# 4. Keep all other columns unchanged (passthrough)
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse_output=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')

In [21]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [31]:
# Fit the ColumnTransformer on the training data and transform it
# Returns the transformed feature array and its shape
transformer.fit_transform(X_train).shape

(80, 7)

In [33]:
# Apply the fitted ColumnTransformer to the test data
# Returns the transformed test feature array and its shape
transformer.transform(X_test).shape

(20, 7)